In [15]:
# This cell is necessary only if you are running on Google Colab. It downloads the files to your
# Colab instance so you don't have to upload them here.

import requests

def save_file(url, file_name):
    r = requests.get(url)
    with open(file_name, 'wb') as f:
      f.write(r.content)

save_file('https://courses.cs.washington.edu/courses/cse416/21sp/homework/hw6/edx_train.csv', 
          'edx_train.csv')
save_file('https://courses.cs.washington.edu/courses/cse416/21sp/homework/hw6/edx_test.csv', 
          'edx_test.csv')

In [17]:
import pandas as pd

# original features and target
features = ['viewed', 
            'explored', 
            'grade', 
            'nevents', 
            'ndays_act', 
            'nplay_video',
            'nchapters', 
            'nforum_posts']
target = ['certified']

# load train and test data
train_data = pd.read_csv('edx_train.csv', na_values=' ')
train_data = train_data.fillna(0)

test_data = pd.read_csv('edx_test.csv', na_values=' ')
test_data = test_data.fillna(0)


# classifier 1: DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# translate the categorical features into numeric ones
train_data_dtc = pd.get_dummies(train_data[features + target])
test_data_dtc = pd.get_dummies(test_data[features])

# prepare the features and target for the model
features_dtc = list(train_data_dtc.columns)
features_dtc.remove('certified')

# train and predict
decision_tree_model = DecisionTreeClassifier(max_depth=6, random_state=6)
decision_tree_model.fit(train_data_dtc[features_dtc], train_data_dtc[target])

predictions_decisiontree = decision_tree_model.predict(test_data_dtc[features_dtc])

# save prediction result
to_save = test_data[['userid_DI']].copy()
to_save.loc[:, 'certified'] = predictions_decisiontree
to_save.to_csv('submission_decisiontree.csv', index=False)


# classifier 2: LogisticRegression
from sklearn.linear_model import LogisticRegression

# train and predict
logitic_regression_model = LogisticRegression(penalty='l2', random_state=1, C=1e23)
logitic_regression_model.fit(train_data[features], train_data[target])

predictions_logistic = logitic_regression_model.predict(test_data[features])

# save prediction result
to_save = test_data[['userid_DI']].copy()
to_save.loc[:, 'certified'] = predictions_logistic
to_save.to_csv('submission_logistic.csv', index=False)



# classifier 3: AdaBoostClassifier
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

# train and predict
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
abc.fit(train_data[features], train_data[target])

predictions_abc = abc.predict(test_data[features])

# save prediction result
to_save = test_data[['userid_DI']].copy()
to_save.loc[:, 'certified'] = predictions_abc
to_save.to_csv('submission_abc.csv', index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example